In [1]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd
import math

In [4]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_dependent.csv")
pi = np.array(pi)
pi = pi.T
A = np.array(A)
b = np.array([1000 for _ in range(m)])  


In [5]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value
print("offline optimal value, ", opv1)


offline optimal value,  42255092.735269636


In [6]:
Anow = A
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)
piNow = np.reshape(piNow, (n,))
bNow = b
bNow = bNow.T

temp = m+n
y = cp.Variable(temp)
e = np.ones(n)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value
p_truth = P
p_truth

array([3254.36583882, 2499.16322271, 2921.66761755, 2894.17251434,
       2981.37443506, 2955.19719493, 3492.75717658, 3141.32743696,
       3019.70152472, 3357.56824267])

In [7]:
def u(s,w):
    value = 0
    for i in range(m):
        value += w/m * cp.log(s[i])
    return value
def learning(k = 100, w = 25):
    A1 = A[:m, :k]
    # pi = np.array(pi)
    pi1 = pi[:,:k]
    x = cp.Variable(k)
    s = cp.Variable(m)
    # print(pi1.shape)
    # print(x.shape)
    obj = cp.Maximize(pi1@x + u(s,w))
    cons = [A1@x + s <= k/n * b, x>=0, x<=1]
    prob = cp.Problem(obj, cons)
    # prob.solve(solver="ECOS")
    prob.solve(solver="SCS")
    opt_sol = x.value
    dual_price = prob.constraints[0].dual_value
    opt_value = prob.value 
    return opt_sol, dual_price, opt_value

def decision(dual_price,k=50):
    # Make the decisions based on the dual prices
    AT = A.T
    xp = []
    xk = []
    for i in range(n):
        cri = (AT[i].dot(dual_price))
        if(pi[0][i] > cri):
            xp.append(1)
        else:
            xp.append(0)  # xt(p) 10000
        if i < k:
            xk.append(0)  #50
    # Check whether the resource constriant is satisfied
    for t in range(k,n):
        flag = 1
        for j in range(m):
            temp1 = A[j,t]*xp[t]
            temp2 = b[j] - (A[j, :t]).dot(np.array(xk))
            if temp1 > temp2:
                flag = 0
                break
        if flag == 1:
            xk.append(xp[t])
        else:
            xk.append(0)
    return xk

**K = 50**

In [8]:
k = 50 
w = 17
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]

ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  50
dual price is [33.97523167 47.80365723 17.04662794  4.66108864 46.68961461  3.90027074
  1.66101362 22.6512888  34.7325194  16.83462047]
objective value is  4701133.0381201
the competetive ratio is  0.11125601043104898


**K = 100**

In [9]:
k = 100
w = 17
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  100
dual price is [34.2889613  34.51483403  0.63133451 40.03179062  7.56905443 88.35475604
 60.05112989 64.04893197 25.37958422 12.88714673]
objective value is  4983592.165841901
the competetive ratio is  0.11794062782122776


**K = 200**

In [10]:
k = 200
w = 25
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  200
dual price is [ 29.98734851 133.65339032 102.61899359 108.90328176  21.91929313
  45.88611008 116.49714492  27.75055694  57.80000283  57.16061925]
objective value is  5474034.090580982
the competetive ratio is  0.12954732166548757


In [8]:
dic_ratio = {}
dic_price = {}
dua_price = []
for k in range(100, 10100, 100):
    opt_sol, dual_price, opt_value = learning(k, 25)
    l_decision = decision(dual_price, k)
    opv2 = (pi.dot(l_decision))[0]
    ratio = opv2/opv1
    dic_ratio[k] = ratio
    dic_price[k] = dual_price
    # print("when k = ", k)
    # print("competetive ratio is", ratio)
    # print("norm of dual price difference", np.linalg.norm(dual_price-p_truth))
    dua_price.append(np.linalg.norm(dual_price-p_truth))

In [10]:
dua_price = pd.DataFrame(np.array(dua_price))
dua_price.to_csv('dua_price.csv',index=False)